This script is used to create the similar/ and unique/ subcorpus of the lipade_groundtruth corpus.

In [13]:
import sys
path = "../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [14]:
from absolute_path import absolutePath
import pandas as pd
import os

In [ ]:
!ls $absolutePath/corpus/lipade_groundtruth

In [ ]:
!mv $absolutePath/corpus/lipade_groundtruth/donnees_IS_corrige.xlsx $absolutePath/corpus/lipade_groundtruth/groundtruth.xlsx

In [ ]:
!mv $absolutePath/corpus/lipade_groundtruth/ground_truth3.0/ $absolutePath/corpus/lipade_groundtruth/similar/

In [ ]:
!mv $absolutePath/corpus/lipade_groundtruth/historicaldataset/ $absolutePath/corpus/lipade_groundtruth/unique/

In [ ]:
corpusPath = absolutePath + 'corpus/lipade_groundtruth/'

similarPath = os.path.join(corpusPath, 'similar/')
uniquePath  = os.path.join(corpusPath, 'unique/')

TODO : Correct manually all errors (add .tif and possibly copy paste the real name) in groundtruth.xlsx and reload the cells :

In [28]:
groundtruth = pd.read_excel(absolutePath + "/corpus/lipade_groundtruth/groundtruth.xlsx", header=None)

groups = [[image for image in group if type(image) is str] for group in groundtruth.values.tolist()]
for g in range(len(groups)):
    for i in range(len(groups[g])):
        if groups[g][i].endswith('.tif') or groups[g][i].endswith('.jpg'):
            groups[g][i] = os.path.join(uniquePath, groups[g][i])
        else:
            groups[g][i] = os.path.join(uniquePath, groups[g][i] + '.jpg')

In [29]:
for g in range(len(groups)):
    for i in range(len(groups[g])):
        if not os.path.exists(groups[g][i]):
            print(g,i,groups[g][i].split("/")[-1],"does not exist")

/ End of the TODO

In [35]:
for g in range(len(groups)):
    for i in range(len(groups[g])):
        if os.path.exists(os.path.join(uniquePath, groups[g][i].split("/")[-1])):
            os.remove(os.path.join(uniquePath, groups[g][i].split("/")[-1]))